# 睡眠段階予測コンペティション チュートリアル
コンペティションでは睡眠ポリグラフ検査によるセンサーデータを利用して睡眠段階を予測します。  

このノートブックでは、睡眠ポリグラフについての簡単な説明とチュートリアルベースラインを紹介します。

このチュートリアルはpythonで脳波を解析するためのライブラリである`MNE`のチュートリアルを参考にしています

https://mne.tools/stable/auto_tutorials/clinical/60_sleep.html#sphx-glr-auto-tutorials-clinical-60-sleep-py

# 1.睡眠ポリグラフ検査
睡眠ポリグラフ検査とは、睡眠関連疾患の診断に用いられる検査の一つです。  
睡眠ポリグラフは、ポリソムノグラフィ(polysomnography: PSG)と呼ばれ、睡眠時における脳波、呼吸、脚の運動、あごの運動、眼球運動（レム睡眠とノンレム睡眠）、心電図、酸素飽和度、胸壁の運動、腹壁の運動などを記録します。  
  
PSGの多くは、睡眠時無呼吸症候群を主体とする、睡眠呼吸障害の診断と治療効果判定のために行われています。潜在患者数は約300万人と推定されている一方で、検査に使用する機器の利用や、睡眠段階を判別するのに高度な訓練が必要となるので、検査需要に追いつていないと指摘があります。[1]

# 2.睡眠段階
睡眠段階の判定は睡眠ポリグラフを解析し、**30秒を1epoch**として、epoch毎に睡眠段階を判定します。  
1epochに異なる睡眠段階が混在する場合は、最も多くの時間を占める睡眠段階を採用します。  

今回使用するデータセットである、`Sleep-EDF Database Expanded`では、RechtschaffenとKales(R&K)による睡眠段階ガイドラインが使用され、睡眠段階を以下の8つのラベルでラベル付けされています。
- Sleep stage W (覚醒状態)
- Sleep stage 1
- Sleep stage 2
- Sleep stage 3
- Sleep stage 4
- Sleep stage R (レム睡眠)
- Sleep stage ? (不明)
- Movement time

一方、米国睡眠医学会(AASM)によるガイドラインでは`Sleep stage 1 ~ 4`を3つに分類しています。[2]    

`NME`のチュートリアルや過去の研究では、R&Kによる睡眠段階分類の`Sleep stage 3`と`Sleep stage 4`を`Sleep stage 3/4`に変更して  
AASMによる睡眠段階分類に合わせています。  


このコンペティションでもその方法にならい、AASMによる分類のラベル付けに変更します。

`Movement time`と`Sleep stage ?`は`sample_submission.csv`の正解データには含まれていません

In [1]:
# ラベル名をIDに置き換える
# Sleep stage 3とSleep stage 4を同じIDとして、AASMによる分類に変更する
RANK_LABEL2ID = {
    'Movement time': -1,
    'Sleep stage ?': -1,
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3': 3,
    'Sleep stage 4': 3,
    'Sleep stage R': 4
}

# AASMによる分類
LABEL2ID = {
    'Movement time': -1,
    'Sleep stage ?': -1,
    'Sleep stage W': 0,
    'Sleep stage 1': 1,
    'Sleep stage 2': 2,
    'Sleep stage 3/4': 3,
    'Sleep stage R': 4
}
ID2LABEL = {v:k for k, v in LABEL2ID.items()}

# ライブラリのインストールとデータ設定

In [2]:
# ライブラリのインストール
# !pip install -q mne==1.2.0

In [52]:
import datetime
from pathlib import Path
from typing import Dict, List
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
import warnings

from sklearn.model_selection import GroupKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score

import mne

In [4]:
# ドライブのマウントとカレントディレクトリの設定
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# ダウンロードしたデータをinput/sleep-stage-detection/*というディレクトリ構成でドライブに保存しています
# ドライブ経由だと処理が遅くなるため、inputデータをランタイム配下にコピーします (エラーが発生する場合があります。その場合はセルを再実行してください)
# !cp -r /content/drive/MyDrive/sleep-cpt/input .

In [6]:
# !ls ./input/sleep-stage-detection

In [7]:
# ファイルの解凍(少し時間がかかります)
# !tar -zxf ./input/sleep-stage-detection/edf_data.tar.gz -C ./input/sleep-stage-detection/

In [10]:
DATA_DIR = Path("../data")
EDF_DIR = DATA_DIR / "edf_data"
SUBMISSION_DIR = Path("../submission")

# sample submissionの確認
はじめにサブミッションの形式を確認します。


In [11]:
sample_submission_df = pd.read_csv(SUBMISSION_DIR/"sample_submission.csv", parse_dates=[1])

In [12]:
sample_submission_df

,id,meas_time,condition
0,53c1555,1989-11-20 23:19:30,Sleep stage W
1,53c1555,1989-11-20 23:20:00,Sleep stage W
2,53c1555,1989-11-20 23:20:30,Sleep stage W
3,53c1555,1989-11-20 23:21:00,Sleep stage W
4,53c1555,1989-11-20 23:21:30,Sleep stage W
...,...,...,...
52291,9b444bb,1989-04-12 07:32:30,Sleep stage W
52292,9b444bb,1989-04-12 07:33:00,Sleep stage W
52293,9b444bb,1989-04-12 07:33:30,Sleep stage W
52294,9b444bb,1989-04-12 07:34:00,Sleep stage W


meas_timeは睡眠段階の予測時間です。  
30秒を1epochとして睡眠段階を判断しているため、30秒毎に行があるのが分かります。

提供されているデータは波形データとアノテーションデータが別々になっているため  
この形式に処理する必要があります。  

# レコードデータの読み込み
被験者のメタデータは`**_recoreds.csv`としてcsvで保存されています。

In [13]:
train_record_df = pd.read_csv(DATA_DIR/"train_records.csv")
test_record_df = pd.read_csv(DATA_DIR/"test_records.csv")

In [14]:
train_record_df.head()

,id,subject_id,night,age,sex,lights_off,psg,hypnogram
0,3c1c5cf,07c46da,1,90,male,23:28:00,3c1c5cf-PSG.edf,3c1c5cf-Hypnogram.edf
1,8fbd71b,07c46da,2,90,male,01:29:00,8fbd71b-PSG.edf,8fbd71b-Hypnogram.edf
2,9d5e9ec,21969ff,1,51,female,23:10:00,9d5e9ec-PSG.edf,9d5e9ec-Hypnogram.edf
3,e0df8c0,21969ff,2,51,female,23:15:00,e0df8c0-PSG.edf,e0df8c0-Hypnogram.edf
4,3e404fc,22b58e8,1,51,female,22:38:00,3e404fc-PSG.edf,3e404fc-Hypnogram.edf


In [15]:
test_record_df.head()

,id,subject_id,night,age,sex,lights_off,psg
0,53c1555,17ca2cd,1,91,female,00:15:00,53c1555-PSG.edf
1,29ef1d5,17ca2cd,2,91,female,23:39:00,29ef1d5-PSG.edf
2,c90b6e7,2c77159,1,56,female,23:55:00,c90b6e7-PSG.edf
3,a61e635,2c77159,2,56,female,00:13:00,a61e635-PSG.edf
4,2cb6860,40dc0bc,1,52,male,23:03:00,2cb6860-PSG.edf


各カラムの説明は以下となります

* id:行に一意のID
* subject_id:被験者にユニークなID
* night:測定した夜のID
* sex:性別
* hypnogram:アノテーションデータのファイル名
* psg:センサーデータのファイル名

In [16]:
# 訓練とテストで被験者は重複していないようです
len(set(train_record_df["subject_id"].unique()) & set(test_record_df["subject_id"].unique()))

0

In [17]:
print("訓練被験者数", len(train_record_df["subject_id"].unique()))
print("テスト被験者数", len(test_record_df["subject_id"].unique()))

訓練被験者数 55
テスト被験者数 23


## 被験者データの表示

被験者(subject)のある晩(night)のデータを見てみましょう。  
`record type`が`PSG`であるものは、`睡眠段階を予測するための波形データ`に相当し、`Hypnogram`は`睡眠段階を記録したデータ`に相当します  

`edf`ファイルは医療時系列データの保存によく使用されるデータフォーマットになります。  
ロードするためには専用のライブラリが必要となります。

このノートブックでは、脳波などを解析する際に用いられるpythonライブラリである`NME`を使用します。  

In [18]:
# パスを設定
train_record_df["hypnogram"] = train_record_df["hypnogram"].map(lambda x: str(EDF_DIR/x))
train_record_df["psg"] = train_record_df["psg"].map(lambda x: str(EDF_DIR/x))
test_record_df["psg"] = test_record_df["psg"].map(lambda x: str(EDF_DIR/x))

In [19]:
row = train_record_df.iloc[0]

### edfファイルの読み込み
`mne.io.read_raw_edf()`メソッドで`edf`ファイルを読み込むことができます。 

**患者一人データを読み込むとcsvで数万行になるため、メモリの対策が必要になります。** 

`NME`ではEDFをファイルを読み込む際に、`preload=False`を設定できます。  
データのメタ情報のみ読み込む機能であり、省メモリで済むようになります。

詳しいリファレンスは以下をご参考下さい。  

https://mne.tools/stable/generated/mne.io.read_raw_edf.html

In [20]:
# edfファイルの読み込み
psg_edf = mne.io.read_raw_edf(row["psg"], preload=False)

Extracting EDF parameters from /Users/satodaichi/Documents/comp/Sleep_Stage_Detection/data/edf_data/3c1c5cf-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [21]:
# 読み込んだデータは、mne.io.edf.edf.RawEDFクラスのインスタンスになります
type(psg_edf)

mne.io.edf.edf.RawEDF

In [22]:
# infoでメタ情報を表示できます
psg_edf.info

<Info | 7 non-empty values
 bads: []
 ch_names: EEG Fpz-Cz, EEG Pz-Oz, EOG horizontal, Resp oro-nasal, EMG ...
 chs: 7 EEG
 custom_ref_applied: False
 highpass: 0.5 Hz
 lowpass: 100.0 Hz
 meas_date: 1989-11-13 16:35:00 UTC
 nchan: 7
 projs: []
 sfreq: 100.0 Hz
>

センサーチャンネルの名前を確認します

In [23]:
psg_edf.ch_names

['EEG Fpz-Cz',
 'EEG Pz-Oz',
 'EOG horizontal',
 'Resp oro-nasal',
 'EMG submental',
 'Temp rectal',
 'Event marker']

それぞれのチャンネルは睡眠ポリグラフからのセンサーチャンネルの情報になっています。  

詳細についてはデータの公開先([Sleep-EDF Database Expanded](https://www.physionet.org/content/sleep-edfx/1.0.0/))をご参照下さい。

日本語でのセンサーの説明などは、参考文献にある睡眠検査[3]や終夜睡眠ポリグラフ[4]をなどを参考にして下さい。

ここでは、センサーチャンネルについて簡単に説明します。  

- EEG： 脳波。すべての予測段階の判定に必要であり、意識水準と対応して変化する。
  - Fpz-Cz、Pz-Ozは頭に取り付けるセンサー箇所の箇所の電位差を表す
- EOG： 眼電図。 眼球運動を測定。
  - horizontalは水平方向に取り付けた電位差を表す
- EMG： オトガイ(顎下)筋電図。
- Resp oro-nasal：口鼻呼吸 (oroが口、nasalが鼻、respはrespirationの略で呼吸を表す)
- Temp rectal：直腸温 (原著に記載がないため、実際に直腸の温度を計測したかどうかは不明)
- Event marker：各時刻で起きたイベントの時刻

読み込んだデータは`to_data_frame`でdataframe化できます。  

In [24]:
psg_df = psg_edf.to_data_frame()

In [25]:
psg_df

,time,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker
0,0.00,84.851770,-11.197558,-219.482051,8.800000e+07,3.234000,1.427420e+07,9.490000e+08
1,0.01,87.957998,13.197558,-215.531868,8.790847e+07,3.233153,1.428125e+07,9.499478e+08
2,0.02,96.862515,-3.908181,-209.112821,8.781710e+07,3.232296,1.428833e+07,9.509124e+08
3,0.03,106.181197,-2.644689,-204.175092,8.772580e+07,3.231429,1.429543e+07,9.518927e+08
4,0.04,102.557265,-25.095971,-198.249817,8.763449e+07,3.230552,1.430254e+07,9.528878e+08
...,...,...,...,...,...,...,...,...
7889995,78899.95,-0.776557,-3.422222,12.097436,6.878336e+07,3.304648,1.428788e+07,9.730920e+08
7889996,78899.96,2.950916,-1.284005,11.109890,6.845655e+07,3.304943,1.428799e+07,9.730803e+08
7889997,78899.97,-12.476679,-0.409280,-1.728205,6.811467e+07,3.305226,1.428810e+07,9.730652e+08
7889998,78899.98,-3.261538,-13.530159,6.172161,6.775786e+07,3.305496,1.428820e+07,9.730468e+08


EEGとEOGは100Hzなので、0.00秒から始まっています。

従って、データの合計時間は(7890000/100)/3600=約22時間です。  

計測時間の開始は`meas_date`で取得できます  
これを設定すると16時から翌14時過ぎまで計測していることが分かります

In [26]:
meas_start = psg_edf.info["meas_date"]
meas_start = meas_start.replace(tzinfo=None)
# 100Hz
psg_df["meas_time"] = pd.date_range(start=meas_start, periods=len(psg_df), freq=pd.Timedelta(1 / 100, unit="s"))

In [27]:
psg_df

,time,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker,meas_time
0,0.00,84.851770,-11.197558,-219.482051,8.800000e+07,3.234000,1.427420e+07,9.490000e+08,1989-11-13 16:35:00.000
1,0.01,87.957998,13.197558,-215.531868,8.790847e+07,3.233153,1.428125e+07,9.499478e+08,1989-11-13 16:35:00.010
2,0.02,96.862515,-3.908181,-209.112821,8.781710e+07,3.232296,1.428833e+07,9.509124e+08,1989-11-13 16:35:00.020
3,0.03,106.181197,-2.644689,-204.175092,8.772580e+07,3.231429,1.429543e+07,9.518927e+08,1989-11-13 16:35:00.030
4,0.04,102.557265,-25.095971,-198.249817,8.763449e+07,3.230552,1.430254e+07,9.528878e+08,1989-11-13 16:35:00.040
...,...,...,...,...,...,...,...,...,...
7889995,78899.95,-0.776557,-3.422222,12.097436,6.878336e+07,3.304648,1.428788e+07,9.730920e+08,1989-11-14 14:29:59.950
7889996,78899.96,2.950916,-1.284005,11.109890,6.845655e+07,3.304943,1.428799e+07,9.730803e+08,1989-11-14 14:29:59.960
7889997,78899.97,-12.476679,-0.409280,-1.728205,6.811467e+07,3.305226,1.428810e+07,9.730652e+08,1989-11-14 14:29:59.970
7889998,78899.98,-3.261538,-13.530159,6.172161,6.775786e+07,3.305496,1.428820e+07,9.730468e+08,1989-11-14 14:29:59.980


## Hypnogramデータ(ラベル)の読み込み
Hypnogramには睡眠段階のラベルが保存されています

In [28]:
# Hypnogramはread_annotationsで読み込むことが可能です
annot = mne.read_annotations(row["hypnogram"])

In [29]:
# 同様にto_data_frameでdataframe化できます
annot_df = annot.to_data_frame()

アノテーションは開始時間とその間隔で構成されています。  


In [30]:
annot_df

,onset,duration,description
0,1970-01-01 00:00:00,16290.0,Sleep stage W
1,1970-01-01 04:31:30,30.0,Sleep stage 1
2,1970-01-01 04:32:00,90.0,Sleep stage 2
3,1970-01-01 04:33:30,1890.0,Sleep stage W
4,1970-01-01 05:05:00,30.0,Sleep stage 1
...,...,...,...
132,1970-01-01 17:37:30,30.0,Sleep stage 1
133,1970-01-01 17:38:00,270.0,Sleep stage W
134,1970-01-01 17:42:30,30.0,Sleep stage 1
135,1970-01-01 17:43:00,15120.0,Sleep stage W


In [31]:
annot_df["description"].value_counts()

Sleep stage 1    47
Sleep stage 2    39
Sleep stage W    28
Sleep stage 3    12
Sleep stage R     8
Sleep stage ?     3
Name: description, dtype: int64

カラムの内容を簡単に説明します
- onset：経過時間（年と日付は設定が必要なので正しくありません。この例では時間の経過のみが正しいものになります）
- duration：アノテーションの間隔時間
- description：睡眠段階のラベル

最初と最後にかなり長い`Sleep Stage W`があることが分かります

## 波形データとアノテーションデータの紐づけ
波形データとアノテーションデータは別々のデータ構造を持っているため、波形データにアノテーションを紐付ける作業が必要になります。

睡眠段階は30秒を1epochとして判定されるため、波形データを30秒毎に区切って1epochとして、`onset`と`duration`から睡眠段階を紐付けます。  
これは煩雑な作業になりますが、`mne`には、この操作を行うメソッドがあります。  

また、前後にかなり長い`Sleep Stage W`があります。  
データ数を減らすため、適当に5時間で解析時間の切り捨てを行います

今回は処理を簡単にするために、チャンネルに`EEG Fpz-Cz`のみを利用します(`read_raw_edf`の`include`メソッドにEEGのみを指定)  

In [32]:
# 再度データを読み込みます
# 簡単のためにEEG Fpz-Czのみ利用します
psg_edf = mne.io.read_raw_edf(row["psg"], include=["EEG Fpz-Cz"], verbose=False)
annot = mne.read_annotations(row["hypnogram"])

# 1時間(3600秒)*5
truncate_start_point = 3600 * 5
truncate_end_point = (len(psg_edf)/100) - (3600 *5)
# 切り捨て
annot.crop(truncate_start_point, truncate_end_point, verbose=False)

# annotationデータを紐づけます
psg_edf.set_annotations(annot, verbose=False, emit_warning=False)

# 30秒毎に分割された睡眠段階
events, _ = mne.events_from_annotations(psg_edf, event_id=RANK_LABEL2ID, chunk_duration=30., verbose=False)

# 30秒毎に1epochとする
tmax = 30. - 1. / psg_edf.info['sfreq']
epoch = mne.Epochs(raw=psg_edf, events=events, event_id=LABEL2ID, tmin=0, tmax=tmax, baseline=None, verbose=False, on_missing='ignore')

# subjectとnightの設定 
epoch.info["temp"] = {
    "id":row["id"],
    "subject_id":row["subject_id"],
    "night":row["night"],
    "truncate_start_point":truncate_start_point,
    "truncate_end_point":truncate_end_point
}

`mne.epochs.Epochs`クラスは波形データにラベルを紐づけたインスタンスになります

In [33]:
print(type(epoch))
epoch

<class 'mne.epochs.Epochs'>


Number of events,1430
Events,Movement time: 20Sleep stage 1: 122Sleep stage 2: 422Sleep stage 3/4: 27Sleep stage ?: 20Sleep stage R: 35Sleep stage W: 804
Time range,0.000 – 29.990 sec
Baseline,off


`RawEDF`クラスと同様にデータフレーム化できます。

データフレームを生成して確認します

In [34]:
epoch_df = epoch.to_data_frame(verbose=False)
epoch_df

,time,condition,epoch,EEG Fpz-Cz
0,0.00,Sleep stage W,0,-80.295971
1,0.01,Sleep stage W,0,-32.252991
2,0.02,Sleep stage W,0,-88.165079
3,0.03,Sleep stage W,0,15.893529
4,0.04,Sleep stage W,0,23.555556
...,...,...,...,...
4289995,29.95,Sleep stage 1,1429,7.092552
4289996,29.96,Sleep stage 1,1429,0.155311
4289997,29.97,Sleep stage 1,1429,6.574847
4289998,29.98,Sleep stage 1,1429,-6.678388


`epoch`のカラムが追加されていることが分かります。  
`epoch`は0.00〜29.99秒の30秒で、1つのepochとなっていることが分かります。  
`time`はsampling rateが100Hzなので0.00秒になっています。  

eventsのarrayは  
1列目が計測開始からの経過時間(10ms)  
3列目はアノテーションのID  
になっています（今回の例では2列目は特に考慮する必要がありません。詳しくは[ドキュメント](https://mne.tools/stable/glossary.html#term-events)をご参考下さい）

In [35]:
events

array([[1800000,       0,       0],
       [1803000,       0,       0],
       [1806000,       0,       0],
       ...,
       [6081000,       0,       0],
       [6084000,       0,       1],
       [6087000,       0,       1]])

In [36]:
# epoch数と一致することが分かります
events.shape

(1430, 3)

# 計測時間の設定

timeカラムは0.00〜29.99秒のエポック内の秒数になっており連続した測定時間になっていません。  
なので計測時間から連続した時間を振ります。


データにはMeasurement date(測定日)の情報がありますが、切り捨てをしたため測定日を計算し直す必要があります。  
具体的には、切り捨てが発生した`truncate_start_point`を`start`として測定日に設定します。  
終了時刻は、データ点の長さ分を100Hzの時間単位で計算すれば決まります。

In [37]:
# 測定日を切り捨てが発生したポイントまでスライド
new_meas_date = epoch.info["meas_date"].replace(tzinfo=None) + datetime.timedelta(seconds=epoch.info["temp"]["truncate_start_point"])
# 連続した時間を作成
epoch_df["meas_time"] = pd.date_range(start=new_meas_date, periods=len(epoch_df), freq=pd.Timedelta(1 / 100, unit="s"))

In [38]:
epoch_df

,time,condition,epoch,EEG Fpz-Cz,meas_time
0,0.00,Sleep stage W,0,-80.295971,1989-11-13 21:35:00.000
1,0.01,Sleep stage W,0,-32.252991,1989-11-13 21:35:00.010
2,0.02,Sleep stage W,0,-88.165079,1989-11-13 21:35:00.020
3,0.03,Sleep stage W,0,15.893529,1989-11-13 21:35:00.030
4,0.04,Sleep stage W,0,23.555556,1989-11-13 21:35:00.040
...,...,...,...,...,...
4289995,29.95,Sleep stage 1,1429,7.092552,1989-11-14 09:29:59.950
4289996,29.96,Sleep stage 1,1429,0.155311,1989-11-14 09:29:59.960
4289997,29.97,Sleep stage 1,1429,6.574847,1989-11-14 09:29:59.970
4289998,29.98,Sleep stage 1,1429,-6.678388,1989-11-14 09:29:59.980


21時から翌の9時までの睡眠の波形データとアノテーションが紐付いたデータが作成されました。  
csvデータとして扱いたい場合、このdataframeを保存しておくとよいかもしれません。  


関数化しておきます。

In [39]:
def epoch_to_df(epoch:mne.epochs.Epochs) -> pd.DataFrame:
    truncate_start_point = epoch.info["temp"]["truncate_start_point"]
    
    df = epoch.to_data_frame(verbose=False)
    
    new_meas_date = epoch.info["meas_date"].replace(tzinfo=None) + datetime.timedelta(seconds=truncate_start_point)
    
    df["meas_time"] = pd.date_range(start=new_meas_date, periods=len(df), freq=pd.Timedelta(1 / 100, unit="s"))
    
    return df

In [40]:
epoch_to_df(epoch)

,time,condition,epoch,EEG Fpz-Cz,meas_time
0,0.00,Sleep stage W,0,-80.295971,1989-11-13 21:35:00.000
1,0.01,Sleep stage W,0,-32.252991,1989-11-13 21:35:00.010
2,0.02,Sleep stage W,0,-88.165079,1989-11-13 21:35:00.020
3,0.03,Sleep stage W,0,15.893529,1989-11-13 21:35:00.030
4,0.04,Sleep stage W,0,23.555556,1989-11-13 21:35:00.040
...,...,...,...,...,...
4289995,29.95,Sleep stage 1,1429,7.092552,1989-11-14 09:29:59.950
4289996,29.96,Sleep stage 1,1429,0.155311,1989-11-14 09:29:59.960
4289997,29.97,Sleep stage 1,1429,6.574847,1989-11-14 09:29:59.970
4289998,29.98,Sleep stage 1,1429,-6.678388,1989-11-14 09:29:59.980


## submission形式への変換
`epoch`毎にグループ化して`time`の最小値を取ることでsubmissionする形式に変換できます

In [41]:
label_df = epoch_df.loc[epoch_df.groupby("epoch")["time"].idxmin()][["meas_time"]].reset_index(drop=True)
label_df["condition"] = "Sleep stage W"
label_df["id"] = epoch.info["temp"]["id"]
label_df

,meas_time,condition,id
0,1989-11-13 21:35:00,Sleep stage W,3c1c5cf
1,1989-11-13 21:35:30,Sleep stage W,3c1c5cf
2,1989-11-13 21:36:00,Sleep stage W,3c1c5cf
3,1989-11-13 21:36:30,Sleep stage W,3c1c5cf
4,1989-11-13 21:37:00,Sleep stage W,3c1c5cf
...,...,...,...
1425,1989-11-14 09:27:30,Sleep stage W,3c1c5cf
1426,1989-11-14 09:28:00,Sleep stage W,3c1c5cf
1427,1989-11-14 09:28:30,Sleep stage W,3c1c5cf
1428,1989-11-14 09:29:00,Sleep stage W,3c1c5cf


関数化しておきます

In [42]:
def epoch_to_sub_df(epoch_df:pd.DataFrame, id, is_train:bool) -> pd.DataFrame:
    cols = ["id", "meas_time"]
    if is_train:
        # 訓練セットの場合はラベルを追加
        cols.append("condition")
    
    label_df = epoch_df.loc[epoch_df.groupby("epoch")["time"].idxmin()].reset_index(drop=True)
    label_df["id"] = id
    
    return label_df[cols]

In [43]:
epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=True)

,id,meas_time,condition
0,3c1c5cf,1989-11-13 21:35:00,Sleep stage W
1,3c1c5cf,1989-11-13 21:35:30,Sleep stage W
2,3c1c5cf,1989-11-13 21:36:00,Sleep stage W
3,3c1c5cf,1989-11-13 21:36:30,Sleep stage W
4,3c1c5cf,1989-11-13 21:37:00,Sleep stage W
...,...,...,...
1425,3c1c5cf,1989-11-14 09:27:30,Sleep stage W
1426,3c1c5cf,1989-11-14 09:28:00,Sleep stage W
1427,3c1c5cf,1989-11-14 09:28:30,Sleep stage W
1428,3c1c5cf,1989-11-14 09:29:00,Sleep stage 1


テストデータにはアノテーションファイルがないので`events`の行列を、先程の例で作成することはできません。  


評価の開始と終了時刻はsample submissionにあるので、前後の切り捨て時刻を計算して求めます。  
アノテーションのラベルIDは`0`としてダミーで生成します。

In [44]:
test_row = test_record_df.iloc[0]
psg_edf = mne.io.read_raw_edf(test_row["psg"], include=["EEG Fpz-Cz"], verbose=False)

# PSGの開始をedfのmeas_dateから取得します
start_psg_date = psg_edf.info["meas_date"]
# 計算のためにtimezoneを消去します
start_psg_date = start_psg_date.replace(tzinfo=None)

# sample submissionから評価される時間レンジを取得します
test_start_time = sample_submission_df[sample_submission_df["id"]==test_row["id"]]["meas_time"].min()
test_end_time = sample_submission_df[sample_submission_df["id"]==test_row["id"]]["meas_time"].max()
# psgの計測開始、評価の対象の開始、評価の対象の終了
print(f"psg start: {start_psg_date},  test start: {test_start_time}, test end: {test_end_time}")

# psgの計測時間から評価の対象の開始と終了を経過時間(秒)になおす
truncate_start_point = int((test_start_time - start_psg_date).total_seconds())
truncate_end_point = int((test_end_time - start_psg_date).total_seconds())+30
print(f"event start second: {truncate_start_point}, event end second: {truncate_end_point} ")

# 30秒毎にデータ点を生成
event_range = list(range(truncate_start_point, truncate_end_point, 30))
events = np.zeros((len(event_range), 3), dtype=int)
events[:, 0] = event_range

# 秒を10m秒に変換する
events = events * 100

psg start: 1989-11-20 15:16:00,  test start: 1989-11-20 23:19:30, test end: 1989-11-21 08:10:30
event start second: 29010, event end second: 60900 


In [45]:
events

array([[2901000,       0,       0],
       [2904000,       0,       0],
       [2907000,       0,       0],
       ...,
       [6081000,       0,       0],
       [6084000,       0,       0],
       [6087000,       0,       0]])

In [46]:
tmax = 30. - 1. / psg_edf.info['sfreq']
epoch = mne.Epochs(raw=psg_edf, events=events, event_id={'Sleep stage W': 0}, tmin=0, tmax=tmax, baseline=None, verbose=False)

In [47]:
epoch.to_data_frame()

Loading data for 1063 events and 3000 original time points ...
0 bad epochs dropped


,time,condition,epoch,EEG Fpz-Cz
0,0.00,Sleep stage W,0,-10.738462
1,0.01,Sleep stage W,0,-13.220513
2,0.02,Sleep stage W,0,-31.481319
3,0.03,Sleep stage W,0,-10.915751
4,0.04,Sleep stage W,0,-39.902564
...,...,...,...,...
3188995,29.95,Sleep stage W,1062,-37.065934
3188996,29.96,Sleep stage W,1062,-34.761172
3188997,29.97,Sleep stage W,1062,-37.331868
3188998,29.98,Sleep stage W,1062,-22.882784


trainとtestで以上の処理をすべて行えるように関数化しておきます

In [53]:
def read_and_set_annotation(record_df:pd.DataFrame, include=None, is_test=False) -> List[mne.epochs.Epochs]:
    whole_epoch_data = []

    for row_id, row in tqdm(record_df.iterrows(), total=len(record_df)):        
        # PSGファイルとHypnogram(アノテーションファイルを読み込む)
        psg_edf = mne.io.read_raw_edf(row["psg"], include=include, verbose=False)
        
        if not is_test:
            # 訓練データの場合
            annot = mne.read_annotations(row["hypnogram"])

            # 切り捨て
            truncate_start_point = 3600 * 5
            truncate_end_point = (len(psg_edf)/100) - (3600 *5)
            annot.crop(truncate_start_point, truncate_end_point, verbose=False)

            # アノテーションデータの切り捨て
            psg_edf.set_annotations(annot, emit_warning=False)
            events, _ = mne.events_from_annotations(psg_edf, event_id=RANK_LABEL2ID, chunk_duration=30., verbose=False)
            
            event_id = LABEL2ID
        else:
            # テストデータの場合
            start_psg_date = psg_edf.info["meas_date"]
            start_psg_date = start_psg_date.replace(tzinfo=None)

            test_start_time = sample_submission_df[sample_submission_df["id"]==row["id"]]["meas_time"].min()
            test_end_time = sample_submission_df[sample_submission_df["id"]==row["id"]]["meas_time"].max()
            
            truncate_start_point = int((test_start_time - start_psg_date).total_seconds())
            truncate_end_point = int((test_end_time- start_psg_date).total_seconds())+30
            
            event_range = list(range(truncate_start_point, truncate_end_point, 30))
            events = np.zeros((len(event_range), 3), dtype=int)
            events[:, 0] = event_range
            events = events * 100
            
            event_id = {'Sleep stage W': 0}
            
    
        # 30秒毎に1epochとする
        tmax = 30. - 1. / psg_edf.info['sfreq']
        epoch = mne.Epochs(raw=psg_edf, events=events, event_id=event_id, tmin=0, tmax=tmax, baseline=None, verbose=False, on_missing='ignore')
        
        # 途中でデータが落ちてないかチェック
        assert len(epoch.events) * 30 == truncate_end_point - truncate_start_point
        
        # メタデータを追加
        epoch.info["temp"] = {
            "id":row["id"],
            "subject_id":row["subject_id"],
            "night":row["night"],
            "age":row["age"],
            "sex":row["sex"],
            "truncate_start_point":truncate_start_point
        }

        whole_epoch_data.append(epoch)

    return whole_epoch_data 

In [54]:
# 処理を簡単にするためにEEG Fpz-Czのみ読み込みます
# またtrainをすべて処理するには少し時間がかかるため、ここでは半分ほどの50を利用することにします
train_record_subset_df = train_record_df.sample(n=50).reset_index(drop=True)

train_subset_epoch = read_and_set_annotation(train_record_subset_df, include=["EEG Fpz-Cz"], is_test=False)
test_whole_epoch = read_and_set_annotation(test_record_df, include=["EEG Fpz-Cz"], is_test=True)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

# epochとlabelの関係の可視化

In [55]:
import plotly.graph_objects as go

In [56]:
sample_events = train_subset_epoch[0].events[:, 2]
sample_epoch_df = train_subset_epoch[0].to_data_frame(verbose=False)

In [57]:
go.Figure(
    data=[
        go.Scatter(x=sample_epoch_df["epoch"].unique(), y=list(map(lambda x: ID2LABEL[x], sample_events)))
    ],
    layout=go.Layout(
        yaxis=dict(title="sleep stage"),
        xaxis=dict(title="epoch"),
    )
)

直感的に分かるように、同じラベルが続いているのでepoch間のLabelには強い相関がありそうです。  
また、切り捨てを行いましたがそれでも前半に長い`Sleep Stage W`があります。切り捨て処理には工夫の余地がありそうです。

センサーデータがどのようになっているか確認してみます。  
すべてをプロットするにはデータが多すぎるため、epochの平均をとります。

In [58]:
epoch_grouped_df = sample_epoch_df.groupby("epoch").agg({"EEG Fpz-Cz":"mean"}).reset_index()

In [59]:
epoch_grouped_df

,epoch,EEG Fpz-Cz
0,0,-0.800732
1,1,-0.714271
2,2,-0.578847
3,3,-0.676026
4,4,-0.731655
...,...,...
1465,1465,-0.528763
1466,1466,-0.288171
1467,1467,-1.049054
1468,1468,-0.433653


In [60]:
go.Figure(
    data=[
        go.Scatter(x=epoch_grouped_df["epoch"], y=epoch_grouped_df["EEG Fpz-Cz"]),
    ],
    layout=go.Layout(
        yaxis=dict(title="EEG Fpz-Cz"),
        xaxis=dict(title="epoch"),
    )
)

中間では脳波の動きは小さく、前後で大きくなっています。  
起床しているときは脳波の動きが活発になっている印象があります  

# 特徴量エンジニアリング
`MNE`ライブラリで紹介されている、パワースペクトル密度を使った方法を試してみます。

https://mne.tools/stable/auto_tutorials/clinical/60_sleep.html#feature-engineering

In [61]:
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5]
        Transformed data. 
    """
    # specific frequency bands
    FREQ_BANDS = {
        "delta": [0.5, 4.5],
        "theta": [4.5, 8.5],
        "alpha": [8.5, 11.5],
        "sigma": [11.5, 15.5],
        "beta": [15.5, 30]
    }

    spectrum = epochs.compute_psd(picks='eeg', fmin=0.5, fmax=30. ,verbose=False)
    psds, freqs = spectrum.get_data(return_freqs=True)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

In [62]:
train_df = []
for epoch in tqdm(train_subset_epoch):
    # 波形をdataframe化
    epoch_df = epoch_to_df(epoch)
    # submit形式のデータフレーム生成
    sub_df = epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=True)
    
    # パワースペクトル密度計算
    feature_df = pd.DataFrame(eeg_power_band(epoch))
    
    _df = pd.concat([sub_df, feature_df], axis=1)
    # 必要ないラベルがある場合は除外する
    _df = _df[~_df["condition"].isin(["Sleep stage ?", "Movement time"])]
    
    train_df.append(_df)
    
train_df = pd.concat(train_df).reset_index(drop=True)

  0%|          | 0/50 [00:00<?, ?it/s]

In [63]:
train_df["condition"].value_counts()

Sleep stage W      34190
Sleep stage 2      22030
Sleep stage R       8532
Sleep stage 1       6445
Sleep stage 3/4     3346
Name: condition, dtype: int64

In [64]:
# ラベルIDに変換
train_df["condition"] = train_df["condition"].map(LABEL2ID)

In [65]:
train_df

,id,meas_time,condition,0,1,2,3,4
0,9cba116,1989-07-10 21:50:00,0,0.004592,0.000569,0.000604,0.000584,0.000587
1,9cba116,1989-07-10 21:50:30,0,0.004012,0.000835,0.000695,0.000606,0.000648
2,9cba116,1989-07-10 21:51:00,0,0.005092,0.000695,0.000545,0.000477,0.000457
3,9cba116,1989-07-10 21:51:30,0,0.004279,0.000913,0.000617,0.000604,0.000572
4,9cba116,1989-07-10 21:52:00,0,0.006599,0.000611,0.000290,0.000225,0.000187
...,...,...,...,...,...,...,...,...
74538,c879128,1989-07-12 08:48:30,0,0.007581,0.000349,0.000099,0.000067,0.000072
74539,c879128,1989-07-12 08:49:00,0,0.007934,0.000253,0.000053,0.000027,0.000022
74540,c879128,1989-07-12 08:49:30,0,0.007978,0.000176,0.000061,0.000040,0.000026
74541,c879128,1989-07-12 08:50:00,0,0.007857,0.000225,0.000075,0.000033,0.000045


In [66]:
test_df = []
for epoch in tqdm(test_whole_epoch):
    # 波形をdataframe化
    epoch_df = epoch_to_df(epoch)
    # submit形式のデータフレーム生成
    sub_df = epoch_to_sub_df(epoch_df, epoch.info["temp"]["id"], is_train=False)

    # パワースペクトル密度計算
    feature_df = pd.DataFrame(eeg_power_band(epoch))
    
    _df = pd.concat([sub_df, feature_df], axis=1)
    
    test_df.append(pd.concat([sub_df, feature_df], axis=1))
    
test_df = pd.concat(test_df)

  0%|          | 0/45 [00:00<?, ?it/s]

# 訓練

検証セットを作成します。  
約20％の被験者を検証セットに割り当てます。  
テストセットの分割のように、同じ被験者は検証セットに含まれないようにします。

ベースラインとしてランダムフォレストを使用します

In [67]:
# 20％の被験者を選ぶ
val_size = int(train_record_df["subject_id"].nunique() * 0.20)
train_all_subjects = train_record_df["subject_id"].unique()
np.random.shuffle(train_all_subjects)

val_subjects = train_all_subjects[:val_size]
val_ids = train_record_df[train_record_df["subject_id"].isin(val_subjects)]["id"]

In [68]:
# 検証セットを作成します
trn_df = train_df[~train_df["id"].isin(val_ids)]
val_df = train_df[train_df["id"].isin(val_ids)]

In [69]:
trn_df

,id,meas_time,condition,0,1,2,3,4
0,9cba116,1989-07-10 21:50:00,0,0.004592,0.000569,0.000604,0.000584,0.000587
1,9cba116,1989-07-10 21:50:30,0,0.004012,0.000835,0.000695,0.000606,0.000648
2,9cba116,1989-07-10 21:51:00,0,0.005092,0.000695,0.000545,0.000477,0.000457
3,9cba116,1989-07-10 21:51:30,0,0.004279,0.000913,0.000617,0.000604,0.000572
4,9cba116,1989-07-10 21:52:00,0,0.006599,0.000611,0.000290,0.000225,0.000187
...,...,...,...,...,...,...,...,...
74538,c879128,1989-07-12 08:48:30,0,0.007581,0.000349,0.000099,0.000067,0.000072
74539,c879128,1989-07-12 08:49:00,0,0.007934,0.000253,0.000053,0.000027,0.000022
74540,c879128,1989-07-12 08:49:30,0,0.007978,0.000176,0.000061,0.000040,0.000026
74541,c879128,1989-07-12 08:50:00,0,0.007857,0.000225,0.000075,0.000033,0.000045


In [70]:
val_df

,id,meas_time,condition,0,1,2,3,4
10294,1d1f6b0,1991-09-26 20:00:00,0,0.007152,0.000674,0.000171,0.000099,0.000077
10295,1d1f6b0,1991-09-26 20:00:30,0,0.007202,0.000711,0.000193,0.000078,0.000054
10296,1d1f6b0,1991-09-26 20:01:00,0,0.006917,0.000868,0.000238,0.000100,0.000074
10297,1d1f6b0,1991-09-26 20:01:30,0,0.006531,0.000761,0.000339,0.000228,0.000154
10298,1d1f6b0,1991-09-26 20:02:00,0,0.006356,0.000810,0.000255,0.000231,0.000205
...,...,...,...,...,...,...,...,...
70177,9da9283,1990-03-20 09:37:30,0,0.006909,0.000524,0.000239,0.000156,0.000155
70178,9da9283,1990-03-20 09:38:00,0,0.006906,0.000860,0.000233,0.000097,0.000081
70179,9da9283,1990-03-20 09:38:30,0,0.007173,0.000688,0.000148,0.000109,0.000069
70180,9da9283,1990-03-20 09:39:00,0,0.007615,0.000420,0.000102,0.000062,0.000044


In [71]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(trn_df.iloc[:, 3:], trn_df["condition"])

RandomForestClassifier(random_state=42)

# 結果

In [72]:
val_preds = model.predict(val_df.iloc[:, 3:])

In [73]:
score = accuracy_score(val_df["condition"], val_preds)
print(score)

0.7135066487374869


In [74]:
print(classification_report(val_df["condition"], val_preds,))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84      6191
           1       0.29      0.10      0.14      1545
           2       0.75      0.81      0.78      3665
           3       0.62      0.44      0.52       581
           4       0.41      0.46      0.43      1404

    accuracy                           0.71     13386
   macro avg       0.57      0.54      0.54     13386
weighted avg       0.68      0.71      0.69     13386



# テストの予測

In [75]:
test_df

,id,meas_time,0,1,2,3,4
0,53c1555,1989-11-20 23:19:30,0.007296,0.000352,0.000198,0.000101,0.000120
1,53c1555,1989-11-20 23:20:00,0.006082,0.000667,0.000296,0.000307,0.000291
2,53c1555,1989-11-20 23:20:30,0.006732,0.000522,0.000260,0.000205,0.000187
3,53c1555,1989-11-20 23:21:00,0.007080,0.000389,0.000206,0.000171,0.000148
4,53c1555,1989-11-20 23:21:30,0.007557,0.000386,0.000133,0.000088,0.000056
...,...,...,...,...,...,...,...
906,9b444bb,1989-04-12 07:32:30,0.007375,0.000615,0.000127,0.000087,0.000044
907,9b444bb,1989-04-12 07:33:00,0.007199,0.000614,0.000158,0.000109,0.000080
908,9b444bb,1989-04-12 07:33:30,0.007683,0.000319,0.000105,0.000058,0.000053
909,9b444bb,1989-04-12 07:34:00,0.007481,0.000402,0.000192,0.000072,0.000065


In [76]:
test_preds = model.predict(test_df.iloc[:, 2:])

In [77]:
sample_submission_df["condition"] = test_preds

In [78]:
sample_submission_df["condition"] = sample_submission_df["condition"].map(ID2LABEL)

In [79]:
sample_submission_df

,id,meas_time,condition
0,53c1555,1989-11-20 23:19:30,Sleep stage W
1,53c1555,1989-11-20 23:20:00,Sleep stage W
2,53c1555,1989-11-20 23:20:30,Sleep stage W
3,53c1555,1989-11-20 23:21:00,Sleep stage W
4,53c1555,1989-11-20 23:21:30,Sleep stage W
...,...,...,...
52291,9b444bb,1989-04-12 07:32:30,Sleep stage W
52292,9b444bb,1989-04-12 07:33:00,Sleep stage W
52293,9b444bb,1989-04-12 07:33:30,Sleep stage W
52294,9b444bb,1989-04-12 07:34:00,Sleep stage W


In [80]:
sample_submission_df["condition"].value_counts()

Sleep stage 2      21355
Sleep stage W      19673
Sleep stage R       5688
Sleep stage 1       3021
Sleep stage 3/4     2559
Name: condition, dtype: int64

In [81]:
sample_submission_df.to_csv(SUBMISSION_DIR / "tutorial_submission.csv", index=False)

#### 改善のヒント
* 利用するチャンネルを増やす
  * REM睡眠は眼球運動を伴います。眼の動きを記録したEOGチャンネルを取り入れることで精度向上の可能性があります
* 特徴量エンジニアリングを工夫する
  * 睡眠段階は前後のepochの睡眠段階と強い相関がありそうなので前後の特徴量を考慮できるようにする
  * 特徴量を自動生成してくれるライブラリ等を利用する
* 最先端の手法をためす
    * [Paper with Code](https://paperswithcode.com/sota/sleep-stage-detection-on-sleep-edf)にSleep EDF Expandedのスコアが掲載されています
    * 深層学習を利用したモデルなども提案されています

# 参考文献
[1] 八木 朝子,我が国における睡眠ポリグラフ検査（PSG）の現状,医学検査,Vol.65,1号,p1-11,2016,[doi:10.14932/jamt.15-70](https://doi.org/10.14932/jamt.15-70)  

[2] 加藤 久美,睡眠関連疾患の評価法,脳と発達,Vol.49,6号,p391-395,2017,[doi:10.11251/ojjscn.49.391](https://doi.org/10.11251/ojjscn.49.391)

[3] 野田 明子,睡眠検査,医学検査,Vol.66,J-STAGE-2号,p.95-105,2017,[doi:10.14932/jamt.17J2-13](https://doi.org/10.14932/jamt.17J2-13)  

[4] 野田 明子,古池 保雄,終夜睡眠ポリグラフ,生体医工学,Vol.46,2号,p.134-143,2008,[doi:10.11239/jsmbe.46.134](https://doi.org/10.11239/jsmbe.46.134)